#### CAUTION !!

・sequence lengthごとにMinMaxScaleをしているため、全体のグラフをプロットすると不自然な形になっている

In [159]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

from Stock_Loader import load_data
from Stock_Preprocessor import process, fast_plot
from Stock_RNN import get_model_0
from Stock_RNN_Embed import get_model_1
from Stock_RNN_Positional import get_model_2
from Stock_Align_Dataset import clean_and_align
from Stock_Utils import *

from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings("ignore")

from importlib import reload

import math

In [234]:
### RELOAD ###

import Stock_Preprocessor
import Stock_Loader
import Stock_RNN
import Stock_RNN_Embed
import Stock_RNN_Positional
import Stock_Align_Dataset
import Stock_Utils

reload(Stock_Preprocessor)
reload(Stock_RNN)
reload(Stock_RNN_Embed)
reload(Stock_RNN_Positional)
reload(Stock_Loader)
reload(Stock_Align_Dataset)
reload(Stock_Utils)

from Stock_Loader import load_data
from Stock_Preprocessor import process, fast_plot
from Stock_RNN import get_model_0
from Stock_RNN_Embed import get_model_1
from Stock_RNN_Positional import get_model_2
from Stock_Align_Dataset import clean_and_align
from Stock_Utils import *

In [8]:
df_appl = load_data("../Datasets/dataset.csv")

...Downloading Datasets
...Adding Values to Datasets
...Done !!


In [9]:
###### DOWNLOADED #######

df_appl.iloc[42364:42366] # 2018-02-15 -> apple revenue hit all time hight

,TimeStamp,Open,High,Low,Close,Volume,Year,Month,Day,Hour,Day_of_Week
42364,2018-02-14 16:00:00,167.32,167.45,167.3,167.37,2.03295e+06,2018,2,14,16,2
42365,2018-02-15 09:30:00,169.79,169.86,169.36,169.84,791331,2018,2,15,9,3


In [10]:
###############
## For Debugging ##
###############

# df_sample = df_appl.iloc[:300]

# df_ran = df_sample.columns.tolist()
# df_ran = df_ran[::-1]
# df_sample = df_sample[df_ran]

# df_sample.head()

In [11]:
###############
## For Debugging ##
###############

# feature = ["Month", "Day_of_Week", "Close", "Open", "High", "Low", "Year"]
# df_sample_aligned, features, categorical_index = clean_and_align(df_sample, feature, debug=True)

In [12]:
###############
## For Debugging ##
###############

# seq_length = 10
# val_length = 60
# test_length = 60

# _train, _train_label, _val, _val_label, _test, _test_label, _test_seq, _meta, _scaler =\
#                                             process(df_sample_aligned, feature, feature_range, seq_length, val_length, test_length, categorical_index)

In [29]:
feature = ["Close", "Open", "High", "Low", "Year", "Month", "Hour", "Day_of_Week"]

df_aligned, feature, categorical_index = clean_and_align(df_appl, feature, debug=True)

...186 row(s) dropped


In [39]:
seq_length = 120
feature_range=(-1,1)
val_length = 600
test_length = 600

train, train_label, val, val_label, test, test_label, test_seq, meta, scaler =\
                                            process(df_aligned, feature, feature_range, seq_length, val_length, test_length, categorical_index)

...Categorical Index Enabled

...Transforming Datasets
...Transforming feature_0
...Transforming feature_1
...Transforming feature_2
...Transforming feature_3
...Transforming feature_4
...Transforming feature_5
...Transforming feature_6
...Transforming feature_7

...Concatenating Datasets

...Done ! Took 1620.9s to complete.


In [121]:
df_check = pd.DataFrame(train[0])
df_check.head(5)

,0,1,2,3,4,5,6,7
0,-1.000000,-0.873418,-1.000000,-0.974684,2017.0,10.0,13.0,0.0
1,-0.823877,-1.000000,-0.944444,-0.998734,2017.0,10.0,13.0,0.0
2,-0.723236,-0.810127,-0.833056,-0.708861,2017.0,10.0,13.0,0.0
3,-0.773556,-0.722025,-0.861111,-0.696203,2017.0,10.0,13.0,0.0
4,-0.949428,-0.772152,-0.972222,-0.924051,2017.0,10.0,13.0,0.0


In [239]:
# RNN
input_size = categorical_index if embed_dim == 1 else (train.shape[2] - categorical_index) * embed_dim + categorical_index
output_size = 4
hidden_dim = 128
n_layers = 2
bidirectional = False
dropout = 0.1

# Time Embedding
embed_dim = 1

# Scheduler
scheduler_step = [75, 100]
gamma = 0.1

# loss_function
lr = 0.001
mse_scaler = True
scale = 0.03

log_dir= "stock_positional"


# model, optimizer, mse_loss, scheduler, writer = get_model_0(input_size, output_size, hidden_dim, n_layers, bidirectional, dropout,
#                                                              lr, scheduler_step, gamma, log_dir)

# model, optimizer, total_loss, scheduler, writer = get_model_1(input_size, output_size, embed_dim, hidden_dim, n_layers, bidirectional, dropout,
#                                                              lr, scheduler_step, gamma, log_dir, categorical_index)

model, optimizer, total_loss, scheduler, writer = get_model_2(input_size, output_size, embed_dim, seq_length, 
                                                      hidden_dim, n_layers, bidirectional, dropout, lr, scheduler_step, gamma, mse_scaler, log_dir, categorical_index, scale)

In [240]:
writer = SummaryWriter(log_dir)

In [241]:
num_epochs = 150
bt_size = 64
acc = math.inf

best_acc = math.inf
best_model = model
loss_accumulated = []
sample = []

In [242]:
for epoch in range(num_epochs):
    
    for phase in ["train", "val"]:
        
        if phase == "train":
            
            model.train()
            
            for i in range(0, len(train)-1, bt_size):
                
                optimizer.zero_grad()
                
                inputs, targets = train[i:i+bt_size, :, :], train_label[i:i+bt_size, :]
                
                if inputs.shape[0] < bt_size: break
                
                inputs = torch.from_numpy(inputs).float()
                targets = torch.from_numpy(targets).float()
                
                outputs = model(inputs)

                loss = total_loss(inputs, targets, outputs)
                loss_accumulated.append(loss.item()*1000)

                loss.backward(retain_graph=True)
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                
                if (i % 3600 == 0) and (i > bt_size):
                    acc = sum(loss_accumulated)*(100) / (len(loss_accumulated))
                    print("....Processing {}th sample out of {} training set. Loss: {}".format(i+1,
                                                                                             len(train), acc))
                    writer.add_scalar("Train_Loss", acc,
                                                      (epoch)*(len(train) // 3600) + (i // 3600))
                    
                optimizer.step()
                scheduler.step()
                
            acc = sum(loss_accumulated)*(100) / (len(loss_accumulated))

            
            print("Epoch: {} Loss: {:.3f}".format(epoch, acc))
            
            loss_accumulated = []
            
            perm = np.random.permutation(train.shape[0])
            train, train_label = train[perm], train_label[perm]
            
        elif phase == "val":
            
            print("...Evaluation Mode")
            
            model.eval()
            
            with torch.no_grad():
                
                for i in range(0, len(val)-1, bt_size):

                    inputs, targets = val[i:i+bt_size,:,:], val_label[i:i+bt_size, :]
                    
                    if inputs.shape[0] < bt_size: break
                    
                    inputs = torch.from_numpy(inputs).float()
                    targets = torch.from_numpy(targets).float()
                    
                    outputs = model(inputs)
                    
                    loss = total_loss(inputs, targets, outputs)
                    loss_accumulated.append(loss.item()*1000)
                
                acc = sum(loss_accumulated)*(100) / (len(loss_accumulated))

                writer.add_scalar("Val_Loss", acc, epoch)
                
                print("Validation: {:.3f}".format(acc))
                
                if acc < best_acc:
                    best_model = model
                    best_acc = acc
                    print("MODEL UPDATED !!")
                
                loss_accumulated = []
                
                perm = np.random.permutation(val.shape[0])
                val, val_label = val[perm], val_label[perm]
                
    print("-" * 70)
    print()

KeyboardInterrupt: 

In [145]:
result_compare(temp_model, test, test_label, [i for i in range(20, 30)], feature, categorical_index)

----------------------------------------------------------------------
idx:  20
                                   	['Close', 'Open', 'High', 'Low', 'Year', 'Month', 'Hour', 'Day_of_Week']

Predicted(y_pred)  :	[0.7847887  0.7903632  0.78251207 0.81544304]
Target(y^)               :	[0.81679389 0.89230769 0.90270796 0.8778626 ]

Last Price(y-1)        :	[0.89312977 0.84615385 0.87838495 0.8778626 ]

(y^ - (y_pred))      => 0.3165650963783264
(y^ - (y-1))             => 0.14681267738342285
----------------------------------------------------------------------
----------------------------------------------------------------------
idx:  21
                                   	['Close', 'Open', 'High', 'Low', 'Year', 'Month', 'Hour', 'Day_of_Week']

Predicted(y_pred)  :	[0.8027873  0.8069143  0.80043125 0.8371036 ]
Target(y^)               :	[0.95419847 0.83076923 0.90270796 0.90839695]

Last Price(y-1)        :	[0.81679389 0.89230769 0.90270796 0.8778626 ]

(y^ - (y_pred))      => 0.348836

In [51]:
# torch.save(best_model, "stock_model_v5")

In [333]:
mse_copy_last_baseline(train, train_label, bt_size)

558.2576291834196

### Validation

分散された位置からとってきた方がいい（今のままだと連続したところからとってきている）

### Embedding

embeddingそのものをどれくらい考慮するかもnetworkでやるべきか

### やること

・　dataをとってくる (screen embedding)
・　loss functionをもうちょいいじる -> チャートだけで大きな変化を予想するのは難しいが、小さな変化をミスってはダメとか
・ positional embedding
・ baysean optimizationでparmeter search
・ transformer

In [1]:
###########################################
###STEP 1 まずEmbeddingなしでClose Priceだけで予測する##
###########################################

In [2]:
##########################################
############STEP 2 Price4つを加える#############
##########################################

In [3]:
##########################################
###########STEP 3 Parametersをいじくる###########
##########################################

In [ ]:
##########################################
########STEP 4 Time Embeddingsを加える##########
##########################################

In [ ]:
##########################################
######STEP 5 Positional Embeddingsを加える#########
##########################################

In [ ]:
##########################################
###########STEP 6 Loss Functionを変える##########
##########################################

In [4]:
##########################################
########STEP 5 Transformerを使って予測する#########
##########################################

In [7]:
##########################################
########STEP 7 Company Embeddingを加える########
##########################################

In [8]:
##########################################
########STEP 8 Day, Hour, MinuteをMixする##########
##########################################